---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

### **4. `RAPTOR`** *(Recursive Abstractive Processing for Tree-Organized Retrieval)*

* **`RAPTOR`** 논문
  * 문서 색인 생성 및 검색에 대한 흥미로운 접근 방식을 제시함
  * [RAPTOR 논문](../12_RAG/data/RAPTOR.pdf)

  * [**`테디노트의 논문 요약글 (노션)`**](https://teddylee777.notion.site/RAPTOR-e835d306fc664dc2ad76191dee1cd859?pvs=4)
    * **`leafs`** = **가장 `low-level` 의 시작 문서 집합** → 이 문서들은 `임베딩`되어 `클러스터링`됨
    * 이후 `클러스터`는 **`유사한 문서들 간의 정보`를 `더 높은 수준`(`더 추상적인`)으로 `요약`**

  * **`leafs`** = 다음과 같이 구성될 수 있음
    * **`단일 문서에서의 텍스트 청크`**  *(≒ 논문 에시)*
    * **`전체 문서`**  *(≒ 아래 예시)*

* `LangChain`의 **`LCEL`** 문서에 적용하기

---

* **`환경설정`**

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()                           # True

In [ ]:
from langsmith import Client
from langsmith import traceable

import os

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음" # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* 셀 출력

    ```bash
    --- LangSmith 환경 변수 확인 ---
    ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    ✅ LangSmith 프로젝트: 'LangChain-prantice'
    ✅ LangSmith API Key: 설정됨
    -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.
    ```

---

#### **1) `데이터 전처리`**

* **`doc`** = **`LCEL` 문서의 고유한 웹페이지**

* **`context`** = 2,000토큰 미만 / 10,000토큰 이상까지 다양

* 웹 문서에서 텍스트 데이터 추출 → 텍스트의 토큰 수 계산 → 히스토그램으로 시각화

  * **`tiktoken` 라이브러리**: 주어진 인코딩 이름에 따라 **`문자열의 토큰 수`를 `계산`**
  * **`RecursiveUrlLoader` 클래스**:
    * 지정된 `URL`에서 웹 문서를 `재귀적으로 로드`
    * 이 과정에서 `BeautifulSoup`를 `활용` → `HTML` 문서에서 `텍스트`를 `추출`
  * `여러 URL`에서 `문서`를 `로드`하여 `모든 텍스트 데이터` **→ `하나의 리스트에 모으기`**
  * `각 문서 텍스트` → **`num_tokens_from_string`** 함수 호출 → 토큰 수 계산 → 리스트에 저장
  * **`matplotlib`** 
    * 계산된 토큰 수의 `분포`를 `히스토그램`으로 `시각화`
    * `토큰 수` = `x축`, 해당 토큰 수를 가진 `문서의 빈도수` = `y축`
  * 히스토그램
    * 데이터의 분포를 이해하는 데 도움
    * 특히 텍스트 데이터의 길이 분포를 시각적으로 파악 가능

In [ ]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup
import tiktoken
import matplotlib.pyplot as plt

# 토큰 수 계산
def num_tokens_from_string(string: str, encoding_name: str):
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens                                               # 11.3s

* 수정 전 코드 - **`LCEL`** 문서 로드

```python

        # LCEL 문서 로드 (수정 전)
        url = "https://python.langchain.com/docs/concepts/lcel/"

        loader = RecursiveUrlLoader(
            url=url, 
            max_depth=20, 
            extractor=lambda x: Soup(x, "html.parser").text
        )
        docs = loader.load()

        print(f"로드된 문서 개수: {len(docs)}")

```

<small>

* 실행 중 `Warning Messages`

<br>

* 메시지 의미
  
  * 1번 메시지 의미: **`"BeautifulSoup" 라이브러리가 HTML 파서(html.parser)를 사용하여 XML 문서처럼 보이는 것을 파싱하고 있음을 알려줌`**
  ```bash

      XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.
  
  ```
  *  * 원인: 웹에서 로드되는 파일 중 일부(예: 웹사이트의 `sitemap.xml` 또는 `일부 API 응답`)가 `HTML`이 아닌 `XML` 형식일 수 있는데, 코드에서 `HTML 파서`를 사용하도록 지정했기 때문 

<br>

  * 2번 메시지 의미: **`XML 파서(lxml 설치 후 features="xml" 사용)를 사용하는 것이 더 안정적일 것이라고 권장`**
  ```bash

          Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable.
  
  ```
  *  * 원인: `안정적인 파싱`을 위해 `더 적합한 도구`를 `사용`하라는 `제안`

<br>

  <small><small><small>

  * 실제 메시지

    ```bash
    /var/folders/h3/l7wnkv352kqftv0t8ctl2ld40000gn/T/ipykernel_94341/3466993954.py:7: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

    Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

    If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

        from bs4 import XMLParsedAsHTMLWarning
        import warnings

        warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

      extractor=lambda x: Soup(x, "html.parser").text
    ```

    ```bash
    /Users/jay/.pyenv/versions/lc_env/lib/python3.13/site-packages/langchain_community/document_loaders/recursive_url_loader.py:44: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

    Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

    If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

        from bs4 import XMLParsedAsHTMLWarning
        import warnings

        warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

      soup = BeautifulSoup(raw_html, "html.parser")
    ```

---

<small>

* 실행이 30분 넘게 진행 중 → 강제 중단

<br>

* 실행이 오래 걸리는 이유: **`RecursiveUrlLoader`** 작동 방식 때문

  * **`재귀적 탐색`**: `RecursiveUrlLoader`는 [기본 URL](https://python.langchain.com/docs/concepts/lcel/) 뿐만 아니라 해당 페이지 내의 `모든 유효한 하위 링크`(`자식 페이지`)를 **`max_depth`** (20으로 설정됨)까지 계속 따라가며 문서를 가져옴

  * **`광범위한 검색`**:
    * `LangChain` 문서 = `매우 방대`하며, `max_depth=20`은 사실상 거의 모든 문서를 긁어오도록 지시하는 것과 같음
    * 이 과정에서 수십, 수백 개의 페이지를 요청하고 `구문 분석`(`Parsing`)해야 하므로 시간이 오래 걸림

  * **`네트워크 지연`**: 각 페이지를 가져올 때마다 네트워크 요청이 발생하며, 이는 로컬 코드 실행보다 훨씬 더 많은 시간을 소모

<br>

* `max_depth`, `url_filter` 조정해보기

  * **속도 문제:** `max_depth=20`이 문제의 주원인 → 원하는 문서의 범위에 맞춰 `max_depth`를 **1** or **2** 등으로 줄여보기
  * **경고 문제:** 
    * `pip install lxml` 실행 → `BeautifulSoup(x, "lxml").text` 사용 → 경고 없이 더 빠르게 파싱 가능

---

* 사전에 `VS Code` 터미널에 설치할 것 
```bash

        pip install lxml

```

---

<small>

* `ReculsiveUrlLoader` 문서 로드 시 `TooManyRedirects` 오류 발생

  * 해당 `URL`이 너무 많은 리디렉션을 발생시키기 때문

  * 웹 페이지 주소가 다른 페이지로 계속해서 자동 연결 → 이 연결 횟수가 `ReculsiveUrlLoader`의 내부 제한 (기본 = 30회)을 초과할 경우 = 오류 발생

<br>

* **`웹사이트 구조나 설정문제 or 해당 페이지가 더이상 유효하지 않아 계속 다른 곳으로 보내지는 경우에 발생할 수 있음`**
  * *해당 페이지 유효함*

<br>

* **`오류 해결 방법`**

  * **`max_depth`** 줄이기 → **`✓`**
    * 교재: `max_depth = 20` → `max_depth = 5`

  * **`url_filter`** 사용
    * 특정 패턴의 `URL`만 포함하거나 제외하도록 매개변수 사용 → 문제가 되는 리디렉션 체인이 시작되는 `URL` 건너뛸 수 있음
    * **`아래에서 시도해보기`**

  * **`문제가 되는 URL 식별 및 제외`**
    * *`ex: https://python.langchain.com/docs/integrations/providers/openai/`*

  * **`다른 로더 사용`**: **`WebBaseLoader`** 등 단일 페이지를 만드는 데 더 적합한 로더를 사용할 수도 있음

  * **`네트워크 or 웹사이트 문제 확인`**

In [ ]:
from langchain_community.document_loaders import RecursiveUrlLoader
from bs4 import BeautifulSoup
import time
import warnings
from bs4 import XMLParsedAsHTMLWarning
import logging

# 경고 메시지 필터링: XMLParsedAsHTMLWarning을 무시하여 콘솔을 깔끔하게 유지
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

# 로깅 설정 (선택 사항: 로더가 어떤 URL을 탐색하는지 확인하려면 주석 해제)
# logging.basicConfig(level=logging.INFO)

print("--- LangChain LCEL 문서 로드 시작 ---")
start_time = time.time()

# 1. 문서 로드 URL 정의
url = "https://docs.langchain.com/oss/python/langchain/overview?_gl=1*y6qerx*_ga*MTQ5MDA0NjU3MC4xNzU5OTI3ODA1*_ga_47WX3HKKY2*czE3NTk5Mjc4MDQkbzEkZzAkdDE3NTk5Mjc4MDQkajYwJGwwJGgw"

# 2. RecursiveUrlLoader 설정
#    - max_depth=2: 로딩 시간을 크게 단축하기 위해 재귀 깊이를 5로 제한
#      (기본 URL + 그 페이지에서 직접 연결된 하위 페이지까지만 탐색)
#    - extractor: lxml 파서를 사용하여 파싱 경고를 제거하고 성능을 개선
loader = RecursiveUrlLoader(
    url=url,
    max_depth=2,                                        # 탐색 깊이를 제한하여 속도 개선
    extractor=lambda x: BeautifulSoup(x, "lxml").text   # 'lxml' 파서를 사용하여 안정성 및 속도 개선
)

try:
    # 3. 문서 로드 실행
    # 이 과정은 max_depth=2로 인해 이전보다 훨씬 빠르게 완료될 것
    docs = loader.load()
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print("-" * 40)
    print("✅ 문서 로드 성공 및 분석 완료!")
    print(f"총 로드된 문서 개수: {len(docs)}")
    print(f"소요 시간: {elapsed_time:.2f}초")
    print("-" * 40)
    
    # 로드된 문서의 첫 번째 문서 일부 출력
    if docs:
        print("\n[첫 번째 문서 미리보기]")
        print(f"소스: {docs[0].metadata.get('source', '알 수 없음')}")
        print(f"내용 (첫 200자): {docs[0].page_content[:200]}...")
except Exception as e:
    print(f"\n❌ 문서 로드 중 예상치 못한 오류 발생: {e}")

<small>

* **`max_depth = 2`** → ⭕️ (`37.5s`)

    ```markdown

    --- LangChain LCEL 문서 로드 시작 ---
    ----------------------------------------
    ✅ 문서 로드 성공 및 분석 완료!
    총 로드된 문서 개수: 36
    소요 시간: 37.55초
    ----------------------------------------

    [첫 번째 문서 미리보기]
    * 소스: `https://docs.langchain.com/oss/python/langchain/overview?_gl=1*y6qerx*_ga*MTQ5MDA0NjU3MC4xNzU5OTI3ODA1*_ga_47WX3HKKY2*czE3NTk5Mjc4MDQkbzEkZzAkdDE3NTk5Mjc4MDQkajYwJGwwJGgw`
    * 내용 (첫 200자): LangChain Overview - Docs by LangChainSkip to main contentOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KOSS (v1-alpha)LangCha...

    ```

In [ ]:
# PydanticOutputParser를 사용한 LCEL 문서 로드 (기본 LCEL 문서 외부)
url = "https://python.langchain.com/docs/introduction/?_gl=1*fiquw4*_gcl_au*MTMwNTA1MDY3NS4xNzU5OTI3ODEy*_ga*MTQ5MDA0NjU3MC4xNzU5OTI3ODA1*_ga_47WX3HKKY2*czE3NTk5Mjc4MDQkbzEkZzEkdDE3NTk5MjgwMTgkajYwJGwwJGgw"

loader = RecursiveUrlLoader(
    url=url, 
    max_depth=1, 
    extractor=lambda x: Soup(x, "html.parser").text
)
docs_pydantic = loader.load()
print(f"로드된 문서 개수: {len(docs_pydantic)}")

<small>

* 로드된 문서 개수: 1  (`0.3s`)

In [ ]:
# Self Query를 사용한 LCEL 문서 로드 (기본 LCEL 문서 외부)
url = "https://docs.langchain.com/oss/python/langchain/overview?_gl=1*1b1sqza*_gcl_au*MTMwNTA1MDY3NS4xNzU5OTI3ODEy*_ga*MTQ5MDA0NjU3MC4xNzU5OTI3ODA1*_ga_47WX3HKKY2*czE3NTk5Mjc4MDQkbzEkZzEkdDE3NTk5Mjc4ODQkajYwJGwwJGgw#core-benefits"

loader = RecursiveUrlLoader(
    url=url, 
    max_depth=1, 
    extractor=lambda x: Soup(x, "html.parser").text
)
docs_sq = loader.load()
print(f"로드된 문서 개수: {len(docs_sq)}")

<small>

* 로드된 문서 개수: 1  (`0.5s`)

In [ ]:
# 문서 텍스트
docs.extend([*docs_pydantic, *docs_sq])
docs_texts = [d.page_content for d in docs]

In [ ]:
print(type(docs_texts))         
print(len(docs_texts))          

<small>

* <class 'list'>
* 38

In [ ]:
# 각 문서에 대한 토큰 수 계산
counts = [num_tokens_from_string(d, "cl100k_base") for d in docs_texts]

print(type(counts))
print(len(counts))

<small>

* <class 'list'>
* 38

---

* **`토큰 수 계산 및 히스토그램`**

In [ ]:
# 토큰 수의 히스토그램 그리기
plt.figure(figsize=(10, 6))
plt.hist(counts, bins=30, color="blue", edgecolor="black", alpha=0.7)
plt.title("Token Counts in LCEL Documents")
plt.xlabel("Token Count")
plt.ylabel("Frequency")
plt.grid(axis="y", alpha=0.75)

# 히스토그램 표시
plt.show

* 문서 텍스트 정렬, 연결 → 토큰 수를 계산하는 과정 설명하기

  * `문서`(`docs`)를 **`메타데이터`** **`"source"`** 키를 기준으로 `정렬`
  * `정렬된 문서 리스트`를 **`역순`** 으로 뒤집기
  * 역순으로 된 문서의 내용을 **`특정 구분자`** (**`"\n\n\n --- \n\n\n"`**)를 사용 → 연결
  * **`num_tokens_from_string`** 함수
    * 연결된 내용의 토큰 수 계산 → 출력
    * 이때, **`"cl100k_base"`** 모델 사용

In [ ]:
# 문서 텍스트 연결하기
# 문서를 출처 메타데이터 기준으로 정렬하기
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])

d_reversed = list(reversed(d_sorted))               # 정렬된 문서를 역순으로 배열

concatenated_content = "\n\n\n --- \n\n\n".join(
    [
        # 역순으로 배열된 문서의 내용을 연결하기
        doc.page_content
        for doc in d_reversed
    ]
)

print(
    "Num tokens in all context: %s"                 # 모든 문맥에서의 토큰 수 출력
    % num_tokens_from_string(concatenated_content, "cl100k_base")
)

<small>

* **`Num tokens in all context: 139556`**  (`0.1s`)

<small>

* 히스토그램  (`0.3s`) + 토큰 수 계산 과정 설명

  * ![히스토그램](../12_RAG/images/output_9.png)

  * 토큰 수 계산 과정 설명: `Num tokens in all context: 139556`

* **`RecursiveCharacterTextSplitter`** → 텍스트를 분할하는 과정 설명하기

  * **`chunk_size_tok`** 변수 설정 → 각 텍스트 `청크의 크기`를 `2000` 토큰으로 지정
  * **`RecursiveCharacterTextSplitter`** **`from_tiktoken_encoder`** 메소드 사용 → 텍스트 분할기 초기화
    * **`청크 크기`(`chunk_size`) = `chunk_size_tok`** 
    * **`청크 간 겹침`(`chunk_overlap`) = `0`**
  * 초기화된 텍스트 분할기의 **`split_text`** 메소드 호출
    * **`concatenated_content`** 변수에 저장된 연결된 텍스트를 분할
    * 분할 결과 = **`texts_split` 변수에 저장**

In [ ]:
# 텍스트 분할을 위한 코드
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size_tok = 2000                               # 토큰의 청크 크기 설정

# 재귀적 문자 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=chunk_size_tok,                      # 청크 크기 설정
    chunk_overlap=0                                 # 청크 중복 허용하지 않음
)

# 주어진 텍스트 분할하기
texts_split = text_splitter.split_text(
    concatenated_content
)                                                   # 0.5s

In [ ]:
print(type(text_splitter))      # <class 'langchain_text_splitters.character.RecursiveCharacterTextSplitter'>

In [ ]:
print(type(texts_split))
print(len(texts_split))

<small>

* <class 'list'>
* 88

---

#### **2) `모델`**

* 다양한 모델 테스트 가능
  
* 교재에서는 `Claude` + `OpenAIEmbeddings`으로 챗봇 모델 구현
  * `OpenAIEmbeddings` 인스턴스화 → `OpenAI`의 임베딩 기능 초기화
  * `LLM` 모델의 `temperature` = `0`으로 설정 → 챗봇 모델 초기화

* **`Cache Embedding`** 사용하기

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# 임베딩 모델 생성하기
# HuggingFace Embeddings 사용
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ hugging-face 임베딩 모델 로딩 완료!")

<small>

* ✅ hugging-face 임베딩 모델 로딩 완료!   (`9.3s`)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# 경고 무시
warnings.filterwarnings("ignore")

# 캐시가 저장될 디렉토리 설정: 절대 경로로 명확히 지정
cache_dir = "/Users/jay/Projects/20250727-langchain-note/12_RAG/cache/"
store = LocalFileStore(cache_dir)
print(f"캐시 저장소 경로 설정 완료: {cache_dir}")

# embeddings 인스턴스 생성하기
embd = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embd, store, namespace=embd.model_name
)

print("✅ Cache Embedding 인스턴스 생성 완료")

<small>

* 캐시 저장소 경로 설정 완료: `/Users/jay/Projects/20250727-langchain-note/12_RAG/cache/`
* ✅ Cache Embedding 인스턴스 생성 완료  (`2.8s`)

---

* **`요약 LLM 초기화하기`**

<small>

* 재귀적 요약 = `LLM` 반복적 호출 → `API` 할당량 초과 → 코드의 무한정 루프에 빠짐

<br>

* **`HuggingFace Pipeline`** 사용하기 *(`local model`)*
  * `BART` 기반 모델: 최대 입력 길이가 비교적 *짧은 편* (`보통 1024 토큰`) → 긴 클러스터 요약에 부적합

```python

      from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
      from transformers import pipeline

      # LLM 대신 요약 파이프라인 설정 (작업 부하가 CPU/로컬로 전환됨)

      summarization_pipeline = pipeline(
          "summarization",
          model="facebook/bart-large-cnn", 
          device=-1,              # CPU 사용 (-1) 또는 GPU 사용 (0)
      )

      llm = HuggingFacePipeline(pipeline=summarization_pipeline)

```

<br>

* 
  * `T5` 모델: *긴 입력 시퀀스 처리에 더 유연*

```python

      from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
      from transformers import pipeline

      # LLM 대신 요약 파이프라인 설정 (작업 부하가 CPU/로컬로 전환됨)

      summarization_pipeline = pipeline(
          "summarization",
          model="t5-base",        # 모델 = T5
          device=-1,              # CPU 사용 (-1) 또는 GPU 사용 (0)
      )

      llm = HuggingFacePipeline(pipeline=summarization_pipeline)

```

In [ ]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model(
    "gemini-2.5-flash", 
    model_provider="google_genai"
    )

print("✅ gemini-2.5-flash 초기화")

<small>

* ✅ gemini-2.5-flash 초기화

    ```bash

    E0000 00:00:1759929438.344679 1220021 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.

    ```

---

#### **3) `트리 구축`**

* **`➀ GMM`**   *(가우시안 혼합 모델)*
  
  * 다양한 `클러스터`에 걸쳐 `데이터 포인트`의 `분포` → `모델링`
  * 모델의 `베이지안 정보 기준`(`BIC`)을 `평가` → **`최적의 클러스터 수` 결정**

* **`➁ UMAP`**   *(Uniform Manifold Approximation and Projection)*
  
  * **`클러스터링 지원`**
  * **`고차원 데이터`의 `차원` `축소`**
  * **`데이터 포인트의 유사성에 기반` → 자연스러운 그룹화를 강조하는 데 도움을 줌**

* **`➂ 지역 및 전역 클러스터링`**
  
  * **`다양한 규모`** 에서 `데이터`를 `분석`하는 데 사용
  * 데이터 내의 `세밀한 패턴`과 `더 넓은 패턴` 모두를 효과적으로 포착

* **`➃ 임계값 설정`**
  
  * `GMM`의 맥락에서 클러스터 멤버십을 결정하기 위해 적용됨
  * `확률 분포`를 기반으로 함: `데이터 포인트`를 **`≥` `1` 클러스터에 할당**

---

* `GMM` 및 `임계값 설정`에 대한 코드 출처: (Sarthi et al)

  * [원본 저장소](https://github.com/parthsarthi03/raptor/blob/master/raptor/cluster_tree_builder.py)
  * [소소한 조정](https://github.com/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-raptor/llama_index/packs/raptor/clustering.py)

* **`global_cluster_embeddings`** 함수 → **`UMAP` 사용** *(임베딩의 글로벌 차원 축소 수행 목적)*

  * **a. `UMAP` 사용**: 입력된 `임베딩`(`embeddings`) → `지정된 차원`(`dim`)으로 **`차원 축소`**

  * **b. `n_neighbors`**
    * 각 포인트를 고려할 **`이웃의 수` 지정** 
    * 제공되지 않을 경우 = 기본 설정 = **`임베딩 수의 제곱근`**

  * **c. `metric`**: `UMAP`에 사용될 **`거리 측정 기준`을 지정**

  * **d. `결과`**: 지정된 차원으로 `축소`된 임베딩이 **`numpy 배열`로 반환**

* 사전에 `VS Code` 터미널에 설치할 것
```bash

        pip install umap-learn

```

In [ ]:
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture

RANDOM_SEED = 42                                    # 재현성을 위한 고정된 시드 값

### --- 위의 인용된 코드에서 주석과 문서화를 추가함 --- ###

def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:

    """
    UMAP을 사용하여 임베딩의 전역 차원 축소를 수행합니다.

    매개변수:
    - embeddings: numpy 배열로 된 입력 임베딩.
    - dim: 축소된 공간의 목표 차원.
    - n_neighbors: 선택 사항; 각 점을 고려할 이웃의 수.
                    제공되지 않으면 임베딩 수의 제곱근으로 기본 설정됩니다.
    - metric: UMAP에 사용할 거리 측정 기준.

    반환값:
    - 지정된 차원으로 축소된 임베딩의 numpy 배열.
    """
    
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)                         # 4.0s

* **`local_cluster_embeddings()`**: 임베딩 데이터에 대해 지역 차원 축소를 수행하는 함수 구현하기

  * 입력된 `임베딩`(`embeddings`) → `UMAP` → `지정된 차원`(`dim`)으로 `차원 축소`
  * 차원 축소 과정: 각 점에 대해 고려할 이웃의 수(`num_neighbors`)와 거리 측정 메트릭(`metric`)을 파라미터로 사용
  * 최종 반환: **`차원이 축소된 임베딩을 numpy 배열`**

In [ ]:
def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    임베딩에 대해 지역 차원 축소를 수행합니다. 이는 일반적으로 전역 클러스터링 이후에 사용됩니다.

    매개변수:
    - embeddings: numpy 배열로서의 입력 임베딩.
    - dim: 축소된 공간의 목표 차원 수.
    - num_neighbors: 각 점에 대해 고려할 이웃의 수.
    - metric: UMAP에 사용할 거리 측정 기준.

    반환값:
    - 지정된 차원으로 축소된 임베딩의 numpy 배열.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings) 

* **`get_optimal_clusters()`** 
  * 주어진 임베딩 데이터를 기반으로 최적의 클러스터 수를 결정하는 데 사용됨
  * **`가우시안 혼합 모델` (`Gaussian Mixture Model`) 사용 → `베이지안 정보 기준` (`Bayesian Information Criterion`, `BIC`) 계산 → 수행됨**

<br>

* 매개변수 설명
  
  * **`입력 임베딩`** (`embeddings`) = **`numpy 배열`**

  * **`최대 클러스터 수`**
    * `max_clusters` = `고려할 클러스터의 최대 수`
    * 기본값 = 50

  * **`random_state` = `42`**
    * 재현성을 위한 난수 상태 = 고정된 값 사용

  * 함수 → 입력 임베딩에 대해 여러 클러스터 수를 시도 → 각각에 대한 BIC 값을 계산

  * **`최소 BIC 값을 가지는 클러스터 수` = `최적의 클러스터 수`로 결정** → 반환

In [ ]:
def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    가우시안 혼합 모델(Gaussian Mixture Model)을 사용하여 베이지안 정보 기준(BIC)을 통해 최적의 클러스터 수를 결정합니다.

    매개변수:
    - embeddings: numpy 배열로서의 입력 임베딩.
    - max_clusters: 고려할 최대 클러스터 수.
    - random_state: 재현성을 위한 시드.

    반환값:
    - 발견된 최적의 클러스터 수를 나타내는 정수.
    """
    
    # 최대 클러스터 수와 임베딩의 길이 중 작은 값을 최대 클러스터 수로 설정
    max_clusters = min(
        max_clusters, len(embeddings)
    )
    
    # 1부터 최대 클러스터 수까지의 범위를 생성
    n_clusters = np.arange(1, max_clusters)
    
    # BIC 점수를 저장할 리스트
    bics = []
    
    # 각 클러스터 수에 대해 반복
    for n in n_clusters:
        # 가우시안 혼합 모델 초기화
        gm = GaussianMixture(
            n_components=n, random_state=random_state, reg_covar=1e-4       # 안정화 매개변수 추가
        )
        gm.fit(embeddings)                      # 임베딩에 대해 모델 학습
        bics.append(gm.bic(embeddings))         # 학습된 모델의 BIC 점수를 리스트에 추가
    return n_clusters[np.argmin(bics)]          # BIC 점수가 가장 낮은 클러스터 수를 반환

* **`GMM_cluster()`** 함수
  * `임베딩` → **`가우시안 혼합 모델`** (`GMM`) 사용 → **`클러스터링`**
  * **`확률 임계값`** 기반

<br>

* 매개변수 설명

  * **`입력된 임베딩`** (`embeddings`) = **`numpy 배열`**

  * **`threshold`** = 임베딩을 `특정 클러스터`에 `할당`하기 위한 `확률 임계값`

  * `random_state` = 결과의 재현성을 위한 시드 값

  * **`get_optimal_clusters` 함수** 호출 = **`최적의 클러스터 수`를 `결정`하기 위함**

  * 결정된 클러스터 수를 바탕으로 `가우시안 혼합 모델`을 초기화 → 입력된 임베딩에 대해 학습 수행함

  * 각 임베딩에 대한 클러스터 `할당 확률 계산` → 이 확률이 주어진 `임계값`을 `초과하는 경우 해당 임베딩을 클러스터에 할당`

  * 최종 반환: **`임베딩의 클러스터 레이블`과 `결정된 클러스터 수`를 `튜플`**

In [ ]:
def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    확률 임계값을 기반으로 가우시안 혼합 모델(GMM)을 사용하여 임베딩을 클러스터링합니다.

    매개변수:
    - embeddings: numpy 배열로서의 입력 임베딩.
    - threshold: 임베딩을 클러스터에 할당하기 위한 확률 임계값.
    - random_state: 재현성을 위한 시드.

    반환값:
    - 클러스터 레이블과 결정된 클러스터 수를 포함하는 튜플.
    """
    
    # 최적의 클러스터 수 구하기
    n_clusters = get_optimal_clusters(embeddings)  
    
    # 가우시안 혼합 모델 초기화하기
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
        
    # 임베딩에 대해 모델 학습하기
    gm.fit(embeddings)
    
    # 임베딩이 각 클러스터에 속할 확률 예측하기
    probs = gm.predict_proba(
        embeddings
    )
    
    # 임계값을 초과하는 확률을 가진 클러스터를 레이블로 선택하기
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters                           # 레이블과 클러스터 수를 튜플로 반환

* **`perform_clustering()`**
  * 임베딩에 대해 차원 축소, 가우시안 혼합 모델을 사용한 글로벌 클러스터링, 그리고 각 글로벌 클러스터 내에서의 로컬 클러스터링을 수행하여 클러스터링 결과를 반환

<br>

* 매개변수 설명

  * **`차원 축소`**:
    * 입력된 임베딩(embeddings) → 차원 축소 수행
    * UMAP 사용 → 지정된 차원(dim)으로 임베딩의 차원을 축소하는 과정 포함

  * **`글로벌 클러스터링 수행`**
    * 차원이 축소된 임베딩 → 가우시안 혼합 모델(GMM) 사용
    * 클러스터 할당은 `주어진 확률 임계값`(`threshold`)을 기준 → 결정

  * **각 글로벌 클러스터 내 `추가적인 로컬 클러스터링 수행`**
    * 글로벌 클러스터링 결과 → 각 글로벌 클러스터에 속한 임베딩들만을 대상으로 **`다시 차원 축소 및 GMM 클러스터링 진행`**

  * 최종 반환 
    * `모든 임베딩`에 대해 글로벌 및 로컬 `클러스터 ID`를 할당
    * 각 임베딩이 속한 클러스터 `ID를 담은 리스트`를 반환 = 임베딩의 순서에 따라 각 임베딩에 대한 `클러스터 ID 배열`을 포함

* 고차원 데이터의 클러스터링을 위해 글로벌 및 로컬 차원에서의 클러스터링을 결합한 접근 방식을 제공 → 더 세분화된 클러스터링 결과를 얻을 수 있으며, 복잡한 데이터 구조를 보다 효과적으로 분석 가능

In [ ]:
def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    임베딩에 대해 차원 축소, 가우시안 혼합 모델을 사용한 클러스터링, 각 글로벌 클러스터 내에서의 로컬 클러스터링을 순서대로 수행합니다.

    매개변수:
    - embeddings: numpy 배열로 된 입력 임베딩입니다.
    - dim: UMAP 축소를 위한 목표 차원입니다.
    - threshold: GMM에서 임베딩을 클러스터에 할당하기 위한 확률 임계값입니다.

    반환값:
    - 각 임베딩의 클러스터 ID를 포함하는 numpy 배열의 리스트입니다.
    """
    if len(embeddings) <= dim + 1:
        # 데이터가 충분하지 않을 때 클러스터링 피하기
        return [np.array([0]) for _ in range(len(embeddings))]

    # 글로벌 차원 축소
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # 글로벌 클러스터링
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # 각 글로벌 클러스터를 순회하며 로컬 클러스터링 수행
    for i in range(n_global_clusters):
        # 현재 글로벌 클러스터에 속하는 임베딩 추출
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # 작은 클러스터는 직접 할당으로 처리
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # 로컬 차원 축소 및 클러스터링
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # 로컬 클러스터 ID 할당, 이미 처리된 총 클러스터 수를 조정
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters

* **`embed()`**: 텍스트 문서의 목록에 대한 임베딩을 생성하는 함수 구현하기

  * 입력= `텍스트 문서`의 `목록`(`texts`)

  * `embd 객체`의 `embed_documents` → 텍스트 문서의 임베딩 생성

  * **`numpy.ndarray` 형태 → 변환 → 반환**

In [ ]:
def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:

    """
    임베딩에 대해 차원 축소, 가우시안 혼합 모델을 사용한 클러스터링, 각 글로벌 클러스터 내에서의 로컬 클러스터링을 순서대로 수행합니다.

    매개변수:
    - embeddings: numpy 배열로 된 입력 임베딩입니다.
    - dim: UMAP 축소를 위한 목표 차원입니다.
    - threshold: GMM에서 임베딩을 클러스터에 할당하기 위한 확률 임계값입니다.

    반환값:
    - 각 임베딩의 클러스터 ID를 포함하는 numpy 배열의 리스트입니다.
    """

    if len(embeddings) <= dim + 1:
        # 데이터가 충분하지 않을 때 클러스터링 피하기
        return [np.array([0]) for _ in range(len(embeddings))]

    # 글로벌 차원 축소
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)

    # 글로벌 클러스터링
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]

    total_clusters = 0

    # 각 글로벌 클러스터를 순회하며 로컬 클러스터링 수행
    for i in range(n_global_clusters):
        # 현재 글로벌 클러스터에 속하는 임베딩 추출
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # 작은 클러스터는 직접 할당으로 처리
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # 로컬 차원 축소 및 클러스터링
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # 로컬 클러스터 ID 할당, 이미 처리된 총 클러스터 수를 조정
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters 

* 텍스트 문서의 목록에 대한 임베딩을 생성하는 함수 **`embed` 구현하기**

  * 입력 = **`텍스트 문서의 목록`** (`texts`)

  * **`embd` 객체의 `embed_documents` 메소드 → 텍스트 문서의 `임베딩 생성`**

  * 생성된 임베딩 = **`numpy.ndarray` 형태로 `변환`하여 `반환`**

In [ ]:
def embed(texts):

    # 텍스트 문서 목록에 대한 임베딩 생성하기
    #
    # 이 함수는 `embd` 객체가 존재한다고 가정하며, 이 객체는 텍스트 목록을 받아 그 임베딩을 반환하는 `embed_documents` 메소드를 가지고 있음
    #
    # 매개변수:
    # - texts: List[str], 임베딩할 텍스트 문서의 목록
    #
    # 반환값:
    # - numpy.ndarray: 주어진 텍스트 문서들에 대한 임베딩 배열
    text_embeddings = embd.embed_documents(
        texts
    )  # 텍스트 문서들의 임베딩 생성하기
    
    # 임베딩을 numpy 배열로 변환하기
    text_embeddings_np = np.array(text_embeddings)  
    
    # 임베딩된 numpy 배열을 반환함
    return text_embeddings_np 

* **`embed_cluster_texts()`**
  * `텍스트 목록`을 `임베딩`하고 `클러스터링` → 원본 텍스트, 해당 임베딩, 그리고 `할당된 클러스터 라벨`을 `포함`하는 **`pandas.DataFrame`을 반환**

<br>

* 매개변수 설명

  * 주어진 텍스트 목록 → `임베딩 생성`

  * **`perform_clustering()`**: 생성된 임베딩 기반 → `클러스터링 수행`

  * **`pandas.DataFrame` 초기화**: 결과 저장 목적

  * **`DataFrame`** = `원본 텍스트`, `임베딩 리스트`, `클러스터 라벨`을 각각 저장함

* 이 함수는 텍스트 데이터의 임베딩 생성과 클러스터링을 하나의 단계로 결합하여, 텍스트 데이터의 구조적 분석과 그룹화를 용이하게 함

In [ ]:
def embed_cluster_texts(texts):

    """
    텍스트 목록을 임베딩하고 클러스터링하여, 텍스트, 그들의 임베딩, 그리고 클러스터 라벨이 포함된 DataFrame을 반환합니다.

    이 함수는 임베딩 생성과 클러스터링을 단일 단계로 결합합니다. 임베딩에 대해 클러스터링을 수행하는 `perform_clustering` 함수의 사전 정의된 존재를 가정합니다.

    매개변수:
    - texts: List[str], 처리될 텍스트 문서의 목록입니다.

    반환값:
    - pandas.DataFrame: 원본 텍스트, 그들의 임베딩, 그리고 할당된 클러스터 라벨이 포함된 DataFrame입니다.
    """

    # 임베딩 생성
    text_embeddings_np = embed(texts)
    
    # 임베딩에 대해 클러스터링 수행
    cluster_labels = perform_clustering(
        text_embeddings_np, 10, 0.1
    )
    
    # 결과를 저장할 DataFrame 초기화
    df = pd.DataFrame()
    
    # 원본 텍스트 저장
    df["text"] = texts
    
    # DataFrame에 리스트로 임베딩 저장
    df["embd"] = list(text_embeddings_np)
    
    # 클러스터 라벨 저장
    df["cluster"] = cluster_labels  
    
    return df

* **`fmt_txt()`**: `pandas`의 `DataFrame`에서 `텍스트 문서`를 `단일 문자열`로 `포맷팅`

  * 입력 파라미터로 `DataFrame`을 받으며, 이 `DataFrame`은 포맷팅할 텍스트 문서를 포함한 `'text'` 컬럼을 가져야 함

  * 모든 텍스트 문서 = `특정 구분자`(`"--- --- \n --- ---"`) 사용 + 연결 → `단일 문자열`로 반환

  * 함수: 연결된 텍스트 문서를 포함하는 `단일 문자열`을 반환

In [ ]:
def fmt_txt(df: pd.DataFrame) -> str:
    """
    DataFrame에 있는 텍스트 문서를 단일 문자열로 포맷합니다.

    매개변수:
    - df: 'text' 열에 포맷할 텍스트 문서가 포함된 DataFrame.

    반환값:
    - 모든 텍스트 문서가 특정 구분자로 결합된 단일 문자열.
    """
    
    # 'text' 열의 모든 텍스트를 리스트로 변환
    unique_txt = df["text"].tolist()
    
    # 텍스트 문서들을 특정 구분자로 결합하여 반환
    return "--- --- \n --- --- ".join(
        unique_txt
    )

* 텍스트 데이터 **`임베딩` → `클러스터링` → 각 클러스터에 대한 `요약`을 `생성`하는 과정** 수행함

  * **`클러스터링 진행`**
    * 주어진 `텍스트 목록`에 대해 `임베딩`을 `생성`하고 `유사성`에 `기반`한 클러스터링을 진행
    * `df_clusters` 데이터프레임을 결과 = `원본 텍스트`, `임베딩`, 그리고 `클러스터 할당 정보`가 포함됨

  * **`데이터프레임 항목 확장`**
    * 클러스터 할당을 쉽게 처리하기 위한 목적
    * 각 행은 `텍스트`, `임베딩`, `클러스터를 포함`하는 `새로운 데이터프레임`으로 `변환`

  * 확장된 데이터프레임에서 **`요약 생성`**
    * `고유한 클러스터 식별자`를 `추출`하고, 각 클러스터에 대한 `텍스트`를 `포맷팅`하여 요약 생성 **→ `df_summary 데이터프레임`에 저장**
    * 각 클러스터의 `요약`, 지정된 `세부 수준`, 그리고 `클러스터 식별자`를 포함

  * 최종 반환
    * 함수 = **`두 개의 데이터프레임`을 포함하는 `튜플`을 반환**
    * 첫 번째 데이터프레임: **`원본 텍스트`, `임베딩`, `클러스터 할당 정보`**
    * 두 번째 데이터프레임: **`각 클러스터에 대한 요약`, `해당 세부 수준`, `클러스터 식별자`**

In [ ]:
def embed_cluster_summarize_texts(
    texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    텍스트 목록에 대해 임베딩, 클러스터링 및 요약을 수행합니다. 이 함수는 먼저 텍스트에 대한 임베딩을 생성하고,
    유사성을 기반으로 클러스터링을 수행한 다음, 클러스터 할당을 확장하여 처리를 용이하게 하고 각 클러스터 내의 내용을 요약합니다.

    매개변수:
    - texts: 처리할 텍스트 문서 목록입니다.
    - level: 처리의 깊이나 세부 사항을 정의할 수 있는 정수 매개변수입니다.

    반환값:
    - 두 개의 데이터프레임을 포함하는 튜플:
        1. 첫 번째 데이터프레임(`df_clusters`)은 원본 텍스트, 그들의 임베딩, 그리고 클러스터 할당을 포함합니다.
        2. 두 번째 데이터프레임(`df_summary`)은 각 클러스터에 대한 요약, 지정된 세부 수준, 그리고 클러스터 식별자를 포함합니다.
    """

    # 텍스트를 임베딩하고 클러스터링하여 'text', 'embd', 'cluster' 열이 있는 데이터프레임 생성하기
    df_clusters = embed_cluster_texts(texts)

    # 클러스터를 쉽게 조작하기 위해 데이터프레임을 확장할 준비하기
    expanded_list = []

    # 데이터프레임 항목을 문서-클러스터 쌍으로 확장하여 처리를 간단하게 하기
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )

    # 확장된 목록에서 새 데이터프레임 생성하기
    expanded_df = pd.DataFrame(expanded_list)

    # 처리를 위해 고유한 클러스터 식별자 검색하기
    all_clusters = expanded_df["cluster"].unique()

    print(f"--Generated {len(all_clusters)} clusters--")

    # 요약
    template = """여기 LangChain 표현 언어 문서의 하위 집합이 있습니다.

    LangChain 표현 언어는 LangChain에서 체인을 구성하는 방법을 제공합니다.

    제공된 문서의 자세한 요약을 제공하십시오.

    문서:
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    
    # llm 삽입
    chain = prompt | model | StrOutputParser()

    # 각 클러스터 내의 텍스트를 요약을 위해 포맷팅하기
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))

    # 요약, 해당 클러스터 및 레벨을 저장할 데이터프레임 생성하기
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )

    return df_clusters, df_summary                          # 1.1s

* 텍스트 데이터를 **`재귀적으로` `임베딩`, `클러스터링` 및 `요약`하는 과정을 구현한 함수**

  * 주어진 텍스트 리스트 = `임베딩`, `클러스터링`, `요약` **→ `각 단계별`로 `결과 저장`**

  * 함수: `최대 지정된 재귀 레벨`까지 실행 or `유일한 클러스터의 수 = 1`이 될 때까지 반복

  * 각 재귀 단계
    * 현재 레벨의 `클러스터링 결과`와 `요약 결과`를 `데이터프레임 형태`로 반환 → 이를 `결과 딕셔너리`에 `저장`
    * 만약 `현재 레벨`이 최대 재귀 레벨보다 `작고`, `유일한 클러스터의 수가 1보다 크다면`: 현재 레벨의 요약 결과를 다음 레벨의 입력 텍스트로 사용하여 재귀적으로 함수를 호출

  * 최종 반환: **`각 레벨별 클러스터 데이터프레임`, `요약 데이터프레임을 포함하는 딕셔너리`**

In [ ]:
def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:

    """
    지정된 레벨까지 또는 고유 클러스터의 수가 1이 될 때까지 텍스트를 재귀적으로 임베딩, 클러스터링, 요약하여
    각 레벨에서의 결과를 저장합니다.

    매개변수:
    - texts: List[str], 처리할 텍스트들.
    - level: int, 현재 재귀 레벨 (1에서 시작).
    - n_levels: int, 재귀의 최대 깊이.

    반환값:
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], 재귀 레벨을 키로 하고 해당 레벨에서의 클러스터 DataFrame과 요약 DataFrame을 포함하는 튜플을 값으로 하는 사전.
    """
    
    # 각 레벨에서의 결과를 저장할 사전
    results = {}  

    # 현재 레벨에 대해 임베딩, 클러스터링, 요약 수행
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

    # 현재 레벨의 결과 저장
    results[level] = (df_clusters, df_summary)

    # 추가 재귀가 가능하고 의미가 있는지 결정
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # 다음 레벨의 재귀 입력 텍스트로 요약 사용
        new_texts = df_summary["summaries"].tolist()
        
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )

        # 다음 레벨의 결과를 현재 결과 사전에 병합
        results.update(next_level_results)

    return results                                              # 0.6s

In [ ]:
# 전체 문서의 개수
len(docs_texts)
print(f"전체 문서의 개수: {len(docs_texts)} 개 ")

<small>

* 전체 문서의 개수: 38 개

In [ ]:
# 트리 구축

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"      # 병렬처리 비활성화

# 문서 텍스트를 리프 텍스트로 설정
leaf_texts = docs_texts

# 재귀적으로 임베딩, 클러스터링 및 요약을 수행하여 결과를 얻음
results = recursive_embed_cluster_summarize(
    leaf_texts, 
    level=1, 
    n_levels=1)                          # n_levels=3 → 1로 축소 (커널 충돌로 인한 지속된 오류)

In [ ]:
print(results)

<small>

* **`--Generated 7 clusters--`**  (`4m 16.1s`)

    ```python

    {1: (                                                 text  \
    0   LangChain Overview - Docs by LangChainSkip to ...   
    1   \n\nhttps://docs.langchain.com/labs/deep-agent...   
    2   Deploy - Docs by LangChainSkip to main content...   
    3   Integration packages - Docs by LangChainSkip t...   
    4   Agents - Docs by LangChainSkip to main content...   
    5   Install LangChain - Docs by LangChainSkip to m...   
    6   LangChain Python v1.0 - Docs by LangChainSkip ...   
    7   Observability - Docs by LangChainSkip to main ...   
    8   Messages - Docs by LangChainSkip to main conte...   
    9   Get started with LangSmith - Docs by LangChain...   
    10  Philosophy - Docs by LangChainSkip to main con...   
    11  Studio - Docs by LangChainSkip to main content...   
    12  Tools - Docs by LangChainSkip to main contentO...   
    13  Test - Docs by LangChainSkip to main contentOu...   
    14  wOF2������ ����| �������������������������...   
    15  Long-term memory - Docs by LangChainSkip to ma...   
    16  Streaming - Docs by LangChainSkip to main cont...   
    17  Human-in-the-loop - Docs by LangChainSkip to m...   
    18  Structured output - Docs by LangChainSkip to m...   
    19  wOF2������0�����������������������������...   
    20  OSS Reference Documentation - Docs by LangChai...   
    21  Models - Docs by LangChainSkip to main content...   
    22  Short-term memory - Docs by LangChainSkip to m...   
    23  LangChain Overview - Docs by LangChainSkip to ...   
    24  Agent Chat UI - Docs by LangChainSkip to main ...   
    25  LangGraph Overview - Docs by LangChainSkip to ...   
    26  Middleware - Docs by LangChainSkip to main con...   
    27  Overview - Docs by LangChainSkip to main conte...   
    28  Context engineering in agents - Docs by LangCh...   
    29  LangChain Overview - Docs by LangChainSkip to ...   
    30  Runtime - Docs by LangChainSkip to main conten...   
    31  Model Context Protocol (MCP) - Docs by LangCha...   
    32  Contributing - Docs by LangChainSkip to main c...   
    33  Retrieval - Docs by LangChainSkip to main cont...   
    34  Quickstart - Docs by LangChainSkip to main con...   
    35  Multi-agent - Docs by LangChainSkip to main co...   
    36  \n\n\n\n\nIntroduction | ü¶úÔ∏èüîó LangChain...   
    37  LangChain Overview - Docs by LangChainSkip to ...   

                                                    embd               cluster  
    0   [-0.053305402398109436, -0.0672820508480072, 0...  [1.0, 1.0, 1.0, 2.0]  
    1   [-0.053877875208854675, -0.024714674800634384,...                 [6.0]  
    2   [-0.005268791224807501, -0.09716880321502686, ...                 [4.0]  
    3   [-0.11690355092287064, -0.06688373535871506, 0...                 [0.0]  
    4   [-0.03355950862169266, -0.08824525028467178, -...                 [6.0]  
    5   [-0.0385858528316021, -0.08613146841526031, 0....                 [2.0]  
    6   [-0.09605161845684052, -0.020263444632291794, ...                 [0.0]  
    7   [-0.03362748771905899, -0.09838922321796417, 0...                 [5.0]  
    8   [-0.027055568993091583, -0.08078095316886902, ...                 [4.0]  
    9   [-0.15380851924419403, -0.08948332816362381, 0...                 [1.0]  
    10  [-0.03396832197904587, -0.12253239005804062, 0...                 [1.0]  
    11  [0.0038095591589808464, -0.095832958817482, 0....                 [6.0]  
    12  [-0.04818306490778923, -0.0709695965051651, 0....                 [4.0]  
    13  [-0.03247443959116936, -0.0807984247803688, 0....                 [4.0]  
    14  [-0.08591422438621521, 0.06414128839969635, -0...                 [6.0]  
    15  [-0.013623809441924095, -0.08108624070882797, ...                 [4.0]  
    16  [-0.03567712381482124, -0.08622419834136963, -...                 [5.0]  
    17  [-0.02739267610013485, -0.06905212253332138, 0...                 [4.0]  
    18  [-0.027303334325551987, -0.05484776571393013, ...                 [4.0]  
    19  [-0.11968187987804413, 0.03897208347916603, -0...                 [6.0]  
    20  [-0.1420479118824005, -0.06697995960712433, 0....                 [0.0]  
    21  [-0.028813177719712257, -0.10515251755714417, ...                 [4.0]  
    22  [0.021889954805374146, -0.07846983522176743, 0...                 [4.0]  
    23  [-0.053305402398109436, -0.0672820508480072, 0...  [1.0, 1.0, 1.0, 2.0]  
    24  [-0.034179240465164185, -0.09057517349720001, ...                 [5.0]  
    25  [-0.09469983726739883, -0.06506861001253128, 0...                 [0.0]  
    26  [-0.054377906024456024, -0.08728640526533127, ...                 [5.0]  
    27  [-0.08649926632642746, -0.029947197064757347, ...                 [3.0]  
    28  [-0.0043255360797047615, -0.1104918122291565, ...                 [6.0]  
    29  [-0.053305402398109436, -0.0672820508480072, 0...  [1.0, 1.0, 1.0, 2.0]  
    30  [-0.0411045141518116, -0.07105415314435959, 0....                 [5.0]  
    31  [-0.028759807348251343, -0.06700382381677628, ...                 [4.0]  
    32  [-0.0884048193693161, -0.047077029943466187, 0...                 [2.0]  
    33  [-0.07178086042404175, -0.05987236276268959, 0...                 [3.0]  
    34  [-0.05516194924712181, -0.08683624863624573, -...                 [5.0]  
    35  [-0.023541731759905815, -0.09582372009754181, ...                 [6.0]  
    36  [-0.0789167732000351, -0.0216425321996212, 0.0...                 [3.0]  
    37  [-0.053305402398109436, -0.0672820508480072, 0...  [1.0, 1.0, 1.0, 2.0]  ,                                            summaries  level  cluster
    0  제공된 문서들은 "LangChain Overview", "Philosophy", "...      1      1.0
    1  제공된 문서들은 'LangChain 표현 언어' (LCEL)에 대한 직접적인 내용을...      1      2.0
    2  제공해주신 문서 하위 집합에는 "LangChain 표현 언어(LangChain Ex...      1      6.0
    3  제공된 문서는 LangChain 표현 언어(LangChain Expression L...      1      4.0
    4  제공된 문서들은 LangChain 표현 언어(LCEL) 자체에 대한 직접적인 정의를...      1      0.0
    5  제공된 문서는 LangChain에서 에이전트를 구축, 배포 및 모니터링하기 위한 다...      1      5.0
    6  제공된 문서는 LangChain 프레임워크와 그 주요 기능, 특히 검색 증강 생성(...      1      3.0)}

    ```

In [ ]:
print(type(results))        # <class 'dict'> 

In [ ]:
print(len(results))         # 1

---

In [ ]:
# 트리 구축_2

# 문서 텍스트를 리프 텍스트로 설정
leaf_texts = docs_texts

# 재귀적으로 임베딩, 클러스터링 및 요약을 수행하여 결과를 얻음
results2 = recursive_embed_cluster_summarize(
    leaf_texts, 
    level=1, 
    n_levels=3)

In [ ]:
print(results2)

<small>

* **`--Generated 6 clusters--`**  (`3m 27.7s`)

* **`--Generated 1 clusters--`**

    ```python

    {1: (                                                 text  \
    0   LangChain Overview - Docs by LangChainSkip to ...   
    1   \n\nhttps://docs.langchain.com/labs/deep-agent...   
    2   Deploy - Docs by LangChainSkip to main content...   
    3   Integration packages - Docs by LangChainSkip t...   
    4   Agents - Docs by LangChainSkip to main content...   
    5   Install LangChain - Docs by LangChainSkip to m...   
    6   LangChain Python v1.0 - Docs by LangChainSkip ...   
    7   Observability - Docs by LangChainSkip to main ...   
    8   Messages - Docs by LangChainSkip to main conte...   
    9   Get started with LangSmith - Docs by LangChain...   
    10  Philosophy - Docs by LangChainSkip to main con...   
    11  Studio - Docs by LangChainSkip to main content...   
    12  Tools - Docs by LangChainSkip to main contentO...   
    13  Test - Docs by LangChainSkip to main contentOu...   
    14  wOF2������ ����| �������������������������...   
    15  Long-term memory - Docs by LangChainSkip to ma...   
    16  Streaming - Docs by LangChainSkip to main cont...   
    17  Human-in-the-loop - Docs by LangChainSkip to m...   
    18  Structured output - Docs by LangChainSkip to m...   
    19  wOF2������0�����������������������������...   
    20  OSS Reference Documentation - Docs by LangChai...   
    21  Models - Docs by LangChainSkip to main content...   
    22  Short-term memory - Docs by LangChainSkip to m...   
    23  LangChain Overview - Docs by LangChainSkip to ...   
    24  Agent Chat UI - Docs by LangChainSkip to main ...   
    25  LangGraph Overview - Docs by LangChainSkip to ...   
    26  Middleware - Docs by LangChainSkip to main con...   
    27  Overview - Docs by LangChainSkip to main conte...   
    28  Context engineering in agents - Docs by LangCh...   
    29  LangChain Overview - Docs by LangChainSkip to ...   
    30  Runtime - Docs by LangChainSkip to main conten...   
    31  Model Context Protocol (MCP) - Docs by LangCha...   
    32  Contributing - Docs by LangChainSkip to main c...   
    33  Retrieval - Docs by LangChainSkip to main cont...   
    34  Quickstart - Docs by LangChainSkip to main con...   
    35  Multi-agent - Docs by LangChainSkip to main co...   
    36  \n\n\n\n\nIntroduction | ü¶úÔ∏èüîó LangChain...   
    37  LangChain Overview - Docs by LangChainSkip to ...   

                                                    embd               cluster  
    0   [-0.053305402398109436, -0.0672820508480072, 0...  [0.0, 0.0, 0.0, 2.0]  
    1   [-0.053877875208854675, -0.024714674800634384,...                 [4.0]  
    2   [-0.005268791224807501, -0.09716880321502686, ...                 [4.0]  
    3   [-0.11690355092287064, -0.06688373535871506, 0...                 [1.0]  
    4   [-0.03355950862169266, -0.08824525028467178, -...                 [4.0]  
    5   [-0.0385858528316021, -0.08613146841526031, 0....                 [2.0]  
    6   [-0.09605161845684052, -0.020263444632291794, ...                 [1.0]  
    7   [-0.03362748771905899, -0.09838922321796417, 0...                 [5.0]  
    8   [-0.027055568993091583, -0.08078095316886902, ...                 [3.0]  
    9   [-0.15380851924419403, -0.08948332816362381, 0...                 [2.0]  
    10  [-0.03396832197904587, -0.12253239005804062, 0...                 [2.0]  
    11  [0.0038095591589808464, -0.095832958817482, 0....                 [4.0]  
    12  [-0.04818306490778923, -0.0709695965051651, 0....                 [3.0]  
    13  [-0.03247443959116936, -0.0807984247803688, 0....                 [3.0]  
    14  [-0.08591422438621521, 0.06414128839969635, -0...                 [4.0]  
    15  [-0.013623809441924095, -0.08108624070882797, ...                 [5.0]  
    16  [-0.03567712381482124, -0.08622419834136963, -...                 [5.0]  
    17  [-0.02739267610013485, -0.06905212253332138, 0...                 [3.0]  
    18  [-0.027303334325551987, -0.05484776571393013, ...                 [3.0]  
    19  [-0.11968187987804413, 0.03897208347916603, -0...                 [4.0]  
    20  [-0.1420479118824005, -0.06697995960712433, 0....                 [1.0]  
    21  [-0.028813177719712257, -0.10515251755714417, ...                 [3.0]  
    22  [0.021889954805374146, -0.07846983522176743, 0...                 [5.0]  
    23  [-0.053305402398109436, -0.0672820508480072, 0...  [0.0, 0.0, 0.0, 2.0]  
    24  [-0.034179240465164185, -0.09057517349720001, ...                 [5.0]  
    25  [-0.09469983726739883, -0.06506861001253128, 0...                 [1.0]  
    26  [-0.054377906024456024, -0.08728640526533127, ...                 [5.0]  
    27  [-0.08649926632642746, -0.029947197064757347, ...                 [0.0]  
    28  [-0.0043255360797047615, -0.1104918122291565, ...                 [4.0]  
    29  [-0.053305402398109436, -0.0672820508480072, 0...  [0.0, 0.0, 0.0, 2.0]  
    30  [-0.0411045141518116, -0.07105415314435959, 0....                 [5.0]  
    31  [-0.028759807348251343, -0.06700382381677628, ...                 [3.0]  
    32  [-0.0884048193693161, -0.047077029943466187, 0...                 [0.0]  
    33  [-0.07178086042404175, -0.05987236276268959, 0...                 [5.0]  
    34  [-0.05516194924712181, -0.08683624863624573, -...                 [5.0]  
    35  [-0.023541731759905815, -0.09582372009754181, ...                 [4.0]  
    36  [-0.0789167732000351, -0.0216425321996212, 0.0...                 [0.0]  
    37  [-0.053305402398109436, -0.0672820508480072, 0...  [0.0, 0.0, 0.0, 2.0]  ,                                            summaries  level  cluster
    0  제공된 문서는 LangChain에 대한 포괄적인 개요를 제공합니다. LangChai...      1      0.0
    1  제공된 문서는 LangChain의 개요, 철학, 설치, 그리고 주요 기능에 대해 설...      1      2.0
    2  제공된 문서 하위 집합은 'LangChain 표현 언어' 자체에 대한 직접적인 자세...      1      4.0
    3  제공된 문서에는 **LangChain 표현 언어(LangChain Expressio...      1      1.0
    4  제공된 문서 하위 집합에는 LangChain 표현 언어(LangChain Expre...      1      5.0
    5  제공된 문서는 LangChain의 핵심 구성 요소와 고급 기능에 대한 포괄적인 정보...      1      3.0), 2: (                                                text  \
    0  제공된 문서는 LangChain에 대한 포괄적인 개요를 제공합니다. LangChai...   
    1  제공된 문서는 LangChain의 개요, 철학, 설치, 그리고 주요 기능에 대해 설...   
    2  제공된 문서 하위 집합은 'LangChain 표현 언어' 자체에 대한 직접적인 자세...   
    3  제공된 문서에는 **LangChain 표현 언어(LangChain Expressio...   
    4  제공된 문서 하위 집합에는 LangChain 표현 언어(LangChain Expre...   
    5  제공된 문서는 LangChain의 핵심 구성 요소와 고급 기능에 대한 포괄적인 정보...   

                                                    embd cluster  
    0  [0.04286152124404907, 0.004999386612325907, 0....     [0]  
    1  [0.022497890517115593, 0.01962566375732422, 0....     [0]  
    2  [0.0048064482398331165, 0.02427043579518795, 0...     [0]  
    3  [-0.03740173578262329, 0.011367842555046082, 0...     [0]  
    4  [0.010943199507892132, 0.038772232830524445, 0...     [0]  
    5  [0.011960526928305626, 0.019280491396784782, 0...     [0]  ,                                            summaries  level  cluster
    0  제공된 문서 하위 집합에는 **LangChain 표현 언어(LangChain Exp...      2        0)}

    ```

In [ ]:
print(type(results2))       # <class 'dict'>
print(len(results2))        # 2

---

* 논문: **`collapsed tree retrieval` = 최고의 성능 보고**

  * `트리 구조`를 `단일 계층`으로 `평탄화` → `모든 노드`에 대해 `동시에` `k-최근접 이웃`(`kNN`) 검색을 적용하는 과정 포함

<br>

* **`Chroma` 벡터 저장소 사용 → 텍스트 데이터의 벡터화 및 검색 가능한 저장소 구축하는 과정**

  * 초기: `leaf_texts`에 저장된 텍스트 데이터 = `all_texts` 변수에 복사

  * `결과 데이터`(`results`)를 `순회` → 각 레벨에서 `요약된 텍스트 추출` → `all_texts`에 `추가`

  * 각 레벨의 `DataFrame`에서 `summaries 컬럼의 값` = `리스트`로 `변환`하여 `추출` → `all_texts`에 `추가`

  * `모든 텍스트 데이터`(`all_texts`) 사용 → `Chroma` 벡터 저장소 구축

    * `Chroma.from_texts()` → `텍스트 데이터 벡터화`, `벡터 저장소 생성`

    * `.as_retriever()` → 검색기(retriever)를 초기화 → 생성된 벡터 저장소를 검색 가능하게 하기 위한 목적

* **`result`**

In [ ]:
from langchain_community.vectorstores import FAISS

# leaf_texts 복사 → all_texts 초기화
all_texts = leaf_texts.copy()

# 각 레벨의 요약 추출 → all_texts에 추가하기 위해 결과를 순회하기
for level in sorted(results.keys()):
    # 현재 레벨의 DataFrame에서 요약 추출하기
    summaries = results[level][1]["summaries"].tolist()
    # 현재 레벨의 요약을 all_texts에 추가하기
    all_texts.extend(summaries)

# all_texts → FAISS vectorstore 구축하기
vectorstore = FAISS.from_texts(
    texts=all_texts, 
    embedding=embd
    )                                           # 1.4s

* **`DB`를 로컬에 저장하기**

In [ ]:
import os

DB_INDEX = "RAPTOR"

# 로컬에 FAISS DB 인덱스가 이미 존재하는지 확인 → 그렇다면 로드하여 vectorstore와 병합한 후 저장하기
if os.path.exists(DB_INDEX):
    local_index = FAISS.load_local(DB_INDEX, embd)
    local_index.merge_from(vectorstore)
    local_index.save_local(DB_INDEX)
    
else:
    vectorstore.save_local(folder_path=DB_INDEX)

In [ ]:
# retriever 생성
retriever = vectorstore.as_retriever()

* **`RAG` 체인 정의** → 특정 코드 예제를 요청하는 방법 구현하기

  * **`hub.pull`** → `RAG 프롬프트` 불러오기

  * **`format_docs()`**
    * 문서 포맷팅을 위해 정의한 함수 
    * 이 함수는 문서의 페이지 내용을 연결하여 반환

  * **`RAG Chain` 구성**
    * `검색기`(`retriever`)로부터 문맥 가져오기 → `format_docs` 함수로 `포맷팅` → `질문 처리`

  * `RunnablePassthrough()` → 질문을 그대로 전달

  * **`체인`**: `프롬프트`, `모델`, `StrOutputParser()` **→ `최종 출력` = `문자열`로 `파싱`**

  * **`rag_chain.invoke`** 메소드 사용 **→ `"How to define a RAG chain? Give me a specific code example."` 라는 질문 처리**

In [ ]:
# 모델(LLM) 생성하기
from langchain.callbacks.base import BaseCallbackHandler
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()

# API 키 확인
if not os.getenv("GOOGLE_API_KEY"):    
    os.environ["GOOGLE_API_KEY"] = input("Enter your Google API key: ")
    
# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",        
    temperature=0,                                              # temperature = 0으로 설정          
    max_output_tokens=4096,
)

class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)

<small>

* `LLM` 생성하기

    ```bash
    E0000 00:00:1759930439.482567 1220021 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
    ```

In [ ]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 생성
prompt = hub.pull("rlm/rag-prompt")

# 문서 포스트 프로세싱
def format_docs(docs):
    # 문서의 페이지 내용을 이어붙여 반환
    return "\n\n".join(doc.page_content for doc in docs)


# RAG 체인 정의
rag_chain = (
    # 검색 결과를 포맷팅하고 질문을 처리
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt                                            # 프롬프트 적용
    | gemini_lc                                         # 모델 적용
    | StrOutputParser()                                 # 문자열 출력 파서 적용
)                                                       # 0.3s

In [ ]:
# Low Level 질문 실행_1
print(rag_chain.invoke("LangChain를 소개해주세요."))

<small>

* `Low Leve 질문 실행_1`  (`6.3s`)

    ```markdown

    LangChain은 대규모 언어 모델(LLM) 기반 애플리케이션, 특히 에이전트를 쉽고 유연하게 구축할 수 있도록 돕는 프레임워크입니다. 이 프레임워크는 다양한 LLM 공급업체의 API를 표준화하고, 외부 데이터 소스 및 도구와의 상호작용을 통해 복잡한 흐름을 오케스트레이션합니다. 또한, LangGraph를 통해 견고한 에이전트 실행을 제공하며 LangSmith를 통해 개발 및 배포 과정에서 디버깅과 모니터링을 지원합니다.

    ```

In [ ]:
# Low Level 질문 실행_2
print(rag_chain.invoke("LangChain agent를 생성하는 예시 코드를 작성해주세요."))

<small>

* `Low Level 질문 실행_2`  (`3.8s`)

    ```markdown
    LangChain 에이전트를 생성하는 예시 코드는 다음과 같습니다. `create_agent` 함수를 사용하여 모델, 도구, 프롬프트를 정의할 수 있습니다.
    ```

    ```python
        from langchain.agents import create_agent

        def get_weather(city: str) -> str:
            """Get weather for a given city."""
            return f"It's always sunny in {city}!"

        agent = create_agent(
            model="anthropic:claude-3-7-sonnet-latest",
            tools=[get_weather],
            prompt="You are a helpful assistant",
        )

        # 에이전트 실행
        agent.invoke(
            {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
        )
    ```

In [ ]:
# Low Level 질문 실행_3
print(rag_chain.invoke("LangChain agent을 사용해 안전하게 개발하기 위한 보안 모범 사례를 안내해주세요"))

<small>

* `Low Level 질문 실행_3`  (`7.6s`)

    ```markdown

    LangChain 에이전트를 안전하게 개발하려면 ReAct 루프를 활용하여 환각을 줄이고 의사결정 과정을 감사 가능하게 만드세요. 또한, 사후-모델 훅을 사용하여 유효성 검사 및 가드레일을 구현하고, 구조화된 출력을 통해 예측 가능한 응답 형식을 강제해야 합니다. 마지막으로, 컨텍스트 엔지니어링을 통해 에이전트의 신뢰성을 높이고, 다중 에이전트 시스템에서는 각 에이전트의 정보 접근을 세밀하게 제어하여 보안을 강화하세요.

    ```

In [ ]:
# Low Level 질문 실행_4
print(rag_chain.invoke("LangChain으로 시작하기에 좋은 것은 무엇인가요?"))

<small>

* `Low Level 질문 실행_4`  (`6.1s`)

    ```markdown

    LangChain은 LLM(대규모 언어 모델) 기반 애플리케이션, 특히 에이전트를 구축하는 가장 쉬운 방법을 제공합니다. 10줄 미만의 코드로 에이전트를 만들 수 있으며, 이는 언어 모델과 도구를 결합하여 작업을 추론하고 해결하는 시스템입니다. 또한, LLM의 한계를 극복하기 위한 검색 증강 생성(RAG) 시스템을 구축하는 데도 활용될 수 있습니다.

    ```

In [ ]:
# Low Level 질문 실행_5
print(rag_chain.invoke("LangChain 프레임워크의 오픈소스 라이브러리에는 어떤 것들이 있으며, 어떤 기능들이 있나요?"))

<small>

* `Low Level 질문 실행_5`   (`7.1s`)

    ```markdown

    LangChain 프레임워크는 LLM 애플리케이션 구축을 위한 핵심 라이브러리인 `langchain`과 `langchain-core`를 포함합니다. `langchain-core`는 채팅 모델 및 기타 구성 요소에 대한 기본 추상화를 제공하며, `langchain`은 체인, 에이전트, 검색 전략 등 애플리케이션의 인지 아키텍처를 구성합니다. 또한, `langchain-community`는 커뮤니티 유지보수 통합을, `langchain-openai`와 같은 통합 패키지들은 다양한 LLM 공급업체와의 연동을 지원하며, LangGraph는 상태 저장 에이전트 구축을 위한 오케스트레이션 프레임워크입니다.

    ```

---

* **`results2`**

In [ ]:
from langchain_community.vectorstores import FAISS

# leaf_texts 복사 → all_texts 초기화
all_texts2 = leaf_texts.copy()

# 각 레벨의 요약 추출 → all_texts2에 추가하기 위해 결과를 순회하기
for level in sorted(results2.keys()):
    # 현재 레벨의 DataFrame에서 요약 추출하기
    summaries2 = results2[level][1]["summaries"].tolist()
    # 현재 레벨의 요약을 all_texts에 추가하기
    all_texts2.extend(summaries2)

# all_texts → FAISS vectorstore 구축하기
vectorstore = FAISS.from_texts(
    texts=all_texts2, 
    embedding=embd
    )                                           # 1.1s

In [ ]:
# DB 로컬에 저장하기

import os

DB_INDEX = "RAPTOR2"

# 로컬에 FAISS DB 인덱스가 이미 존재하는지 확인 → 그렇다면 로드하여 vectorstore와 병합한 후 저장하기
if os.path.exists(DB_INDEX):
    local_index = FAISS.load_local(DB_INDEX, embd)
    local_index.merge_from(vectorstore)
    local_index.save_local(DB_INDEX)
    
else:
    vectorstore.save_local(folder_path=DB_INDEX)

In [ ]:
# retriever 생성
retriever2 = vectorstore.as_retriever()

* `llm = gemini_lc`, `rag_chain` 그대로 사용

In [ ]:
# RAG 체인 정의
rag_chain2 = (
    # 검색 결과를 포맷팅하고 질문을 처리
    {"context": retriever2 | format_docs, "question": RunnablePassthrough()}
    | prompt                                            # 프롬프트 적용
    | gemini_lc                                         # 모델 적용
    | StrOutputParser()                                 # 문자열 출력 파서 적용
) 

In [ ]:
# Low Level 질문 실행_1
print(rag_chain2.invoke("LangChain를 소개해주세요."))

<small>

* `Low Level 질문 실행_1`  (`5.6s`)

    ```markdown

    LangChain은 LLM(대규모 언어 모델)을 활용하여 애플리케이션, 특히 "에이전트"를 쉽게 구축할 수 있도록 돕는 프레임워크입니다. 이 프레임워크는 다양한 LLM 제공업체의 인터페이스를 표준화하고, LLM을 외부 데이터 및 계산과 결합하여 복잡한 작업을 수행하는 에이전트 중심 설계를 지원합니다. LangGraph를 기반으로 내구성 있는 실행, 스트리밍 등의 고급 기능을 제공하며, LangSmith와 통합되어 에이전트의 디버깅 및 평가를 용이하게 합니다.

    ```

In [ ]:
# Low Level 질문 실행_2
print(rag_chain2.invoke("LangChain agent를 생성하는 예시 코드를 작성해주세요."))

<small>

* `Low Level 질문_실행2`  (`4.0s`)

    ```markdown
    LangChain 에이전트를 생성하려면 `langchain.agents`에서 `create_agent`를 임포트합니다. 모델, 도구 목록, 그리고 프롬프트를 지정하여 에이전트를 초기화할 수 있습니다. 다음은 날씨 정보를 가져오는 도구를 사용하는 예시 코드입니다:
    ```

    ```python

        from langchain.agents import create_agent

        def get_weather(city: str) -> str:
            """Get weather for a given city."""
            return f"It's always sunny in {city}!"

        agent = create_agent(
            model="anthropic:claude-3-7-sonnet-latest",
            tools=[get_weather],
            prompt="You are a helpful assistant",
        )
        agent.invoke({"messages": [{"role": "user", "content": "what is the weather in sf"}]})

    ```

In [ ]:
# Low Level 질문 실행_3
print(rag_chain2.invoke("LangChain agent을 사용해 안전하게 개발하기 위한 보안 모범 사례를 안내해주세요"))

<small>

* `Low Level 질문 실행_3`  (`11.4s`)

    ```markdown

    LangChain v1은 현재 개발 중이며 프로덕션 사용이 권장되지 않으므로, API 변경 및 문서 불완전성에 유의해야 합니다. 안전한 개발을 위해 LangGraph 기반의 LangChain 에이전트는 'Human-in-the-loop' 기능을 제공하며, LangSmith를 통해 에이전트 동작을 디버깅하고 관찰하여 잠재적 문제를 식별할 수 있습니다. 또한, 구문 분석 오류 및 도구 실행 실패에 대한 개선된 오류 처리가 안전성을 높입니다.

    ```

In [ ]:
# Low Level 질문 실행_4
print(rag_chain2.invoke("LangChain으로 시작하기에 좋은 것은 무엇인가요?"))

<small>

* `Low Level 질문 실행_4`  (`5.5s`)

    ```markdown

    제공된 문서에 따르면, LangChain으로 시작하기에 좋은 방법은 **퀵스타트(Quickstart)** 가이드를 활용하는 것입니다. 이 가이드는 기본적인 에이전트와 실제 날씨 예측 에이전트를 구축하는 단계별 과정을 제공합니다. 여기에는 시스템 프롬프트 정의, 도구 생성, 모델 구성, 메모리 추가 및 에이전트 실행 과정이 포함됩니다.

    ```

In [ ]:
# Low Level 질문 실행_4.2
print(rag_chain2.invoke("LangChain 퀵스타트 과정을 더 자세히 알려주세요"))

In [ ]:
# Low Level 질문 실행_5
print(rag_chain2.invoke("LangChain 프레임워크의 오픈소스 라이브러리에는 어떤 것들이 있으며, 어떤 기능들이 있나요?"))

<small>

* `Low Level 질문 실행_5`  (`16.5s`)

    ```markdown

    LangChain은 LLM 기반 에이전트 애플리케이션 구축을 위한 오픈소스 프레임워크이며, LangGraph는 장기 실행 상태 저장 에이전트 오케스트레이션을 위한 저수준 오픈소스 라이브러리입니다. LangChain Python은 1000개 이상의 광범위한 통합 생태계를 제공하며, LangGraph는 내구성 있는 실행, 스트리밍, 휴먼-인-더-루프, 포괄적인 메모리 기능을 지원합니다. 이 프레임워크들은 관찰 가능성, 단기/장기 기억 관리, 미들웨어 제어, 외부 지식 검색(RAG) 등 다양한 기능을 제공하여 지능형 에이전트 개발을 돕습니다.

    ```

In [ ]:
# Low Level 질문 실행_6
print(rag_chain2.invoke("LangChain의 채팅 모델의 종류를 알려주세요"))

<small>

* `Low Level 질문 실행_6`  (`8.2s`)

    ```markdown

    LangChain은 다양한 LLM 제공업체의 채팅 모델을 지원합니다. 이러한 제공업체에는 OpenAI, Anthropic, Google, AWS, Hugging Face 등이 포함됩니다. LangChain은 이러한 다양한 모델의 고유한 API와 응답 형식을 표준화하여 개발자가 쉽게 교체할 수 있도록 돕습니다.

    ```

In [ ]:
# Low Level 질문 실행_7
print(rag_chain2.invoke("LangChain Python API Reference가 무엇인지, 어떤 종류가 있는지 알려주세요"))

<small>

* `Low Level 질문 실행_7`  (`5.3s`)

    ```markdown

    LangChain Python API Reference는 LangChain Python 라이브러리에 대한 포괄적인 API 참조 문서입니다. 여기에는 채팅 모델, 도구, 에이전트, 그래프 API, 상태 관리, 체크포인팅 등 다양한 구성 요소가 포함됩니다. 또한 OpenAI, Anthropic, Google 등 1000개 이상의 통합 패키지를 통해 채팅 및 임베딩 모델, 도구 및 툴킷, 문서 로더, 벡터 저장소 등을 지원합니다.

    ```

---

* next: ***`05. 대화내용을 기억하는 RAG 체인`***

---